In [6]:
import re
import os
current_dir = os.getcwd()
print(current_dir)

YOUR/Current/directory


In [8]:
INPUT_FILE = "bibtex_in.bib"
OUTPUT_FILE = "bibtex_out.json"

json_file = []
# jsonファイルの書き出し
LINES_BEGIN = ['{', '"pages": [']
json_file.extend(LINES_BEGIN)
# jsonファイルの書き終わり
LINES_END = [']', '}']
# 各Scrapboxページの書き出し
PAGE_PREFIX = []
# 各Scrapboxページの書き終わり
PAGE_SUFFIX = ["[* Tag:]", "#read2021 #read202103"]

# 全ての要素から取り除く文字列(正規表現)
REMOVE_FROM_ALL = '["\{\}\[\]]'
# 外部リンク(https)を含む要素から取り除く文字列(正規表現)
REMOVE_FROM_LINK = '["\[\]]'
REPLACE_IN_LINK = '[\{\}]'
# 論文タイトルから取り除く文字列(正規表現)
REMOVE_FROM_TITLE = "[\{\}]"
# Scrapbox上でリンク化したいもの
LINK_ITEMS = ["author", "journal", "archivePrefix", "year"]
# jsonファイルでエスケープが必要なもの
# JSON_ESCAPE = ['\\','"', '/', '\b', '\f', '\n', '\r', '\t', '\a']
JSON_ESCAPE = {'\\':'\\\\','"':'\"', '\b':'\\b', '\f':'\\f', '\n':'\\n', '\r':'\\r', '\t':'\\t', '\a':'\\a'}

path = os.path.join(current_dir, INPUT_FILE)
print("importing papers from : {}".format(path))

lines = []
with open(path, 'r') as f:
    lines = f.readlines()

papers = []
new_paper = ""
for num_line, line in enumerate(lines):
    if line[0] == "@":
        if len(new_paper) > 0:
            papers.append(new_paper)
        new_paper = line.strip()
    elif num_line == len(lines)-1:
        new_paper += line.strip()
        papers.append(new_paper)
    else:
        if not len(new_paper)==0:
            new_paper += line.strip() + " "
    

print("number of papers being imported: {}".format(len(papers)))

importing papers from : YOUR/Current/directory/bibtex_in.bib
number of papers being imported: 4


In [10]:
for num_paper, paper in enumerate(papers):
    paper_converted = []
    item = re.search(",[\d\s\w]+=", paper)
    item_label = paper[item.start()+1:item.end()-1].strip()
    paper_converted.append(paper[:item.start()])
    paper = paper[item.end():].strip()

    
    last_flg = 0
    title = ""
    while re.search(",[\d\s\w]+=", paper) or last_flg == 1:

        item = re.search(",[\d\s\w]+=", paper)
        if item:
            item_sequence = paper[:item.start()]
        elif last_flg==1:
            item_sequence = paper.strip()[:-1].strip()
        else:
            assert False
        
        if "https" in item_sequence:
            item_sequence = re.sub(REMOVE_FROM_LINK, "", item_sequence)
            item_sequence = re.sub(REPLACE_IN_LINK, " ", item_sequence)
        else:
            item_sequence = re.sub(REMOVE_FROM_ALL, "", item_sequence)


        # 論文タイトル
        if item_label.lower() == "title":
            item_sequence = re.sub(REMOVE_FROM_TITLE, "", item_sequence)
            title = item_sequence

        # 著者
        if item_label.lower() == "author":
            author_names = re.split(" and ", item_sequence)
            if "author" in LINK_ITEMS:
                for i, author_name in enumerate(author_names):
                    #if i==0 or i==-1:
                    #    author_name = re.sub('"', '', author_name)
                    author_names[i] = "[" + author_name + "]"
            item_sequence = " and ".join(author_names)

        # その他Scrapboxでリンク化するもの
        if item_label.lower() in LINK_ITEMS and not item_label.lower() == "author":
            item_sequence = "[" + item_sequence + "]"
        
        # エスケープ記号を追加
        for escape in JSON_ESCAPE.keys():
            if escape in item_sequence:
                item_sequence = list(item_sequence)
                for index, character in enumerate(item_sequence):
                    if character in JSON_ESCAPE.keys():
                        item_sequence[index] = JSON_ESCAPE[character]
                item_sequence = "".join(item_sequence)
            if escape in item_label:
                item_label = list(item_label)
                for index, character in enumerate(item_label):
                    if character in JSON_ESCAPE.keys():
                        item_label[index] = JSON_ESCAPE[character]
                item_label = "".join(item_label)

        # 保存
        paper_converted.append(item_label + "\\t= " + item_sequence)

        if item:
            item_label = paper[item.start()+1:item.end()-1].strip()
            paper = paper[item.end():].strip()

        if not re.search(",[\d\s\w]+=", paper) and last_flg==0:
            last_flg = 1
        elif last_flg == 1:
            last_flg = 0

    # タイトルにエスケープ記号を追加
    for escape in JSON_ESCAPE.keys():
        if escape in title:
            title = list(title)
            for index, character in enumerate(title):
                if character in JSON_ESCAPE.keys():
                    title[index] = JSON_ESCAPE[character]
            title = "".join(title)

    # paperごとにjson_fileへ追加
    json_file.append('{')
    json_file.append('"title": "{}",'.format(title))
    json_file.append('"lines": [')
    json_file.append('"{}",'.format(title)) #一行目は必ずタイトルと一致
    for prefix in PAGE_PREFIX:
        json_file.append('"{}",'.format(prefix))
    for line in paper_converted:
        json_file.append('"{}",'.format(line))
    json_file.append('"}",')
    for suffix in PAGE_SUFFIX:
        json_file.append('"{}",'.format(suffix))
    json_file[-1] = json_file[-1][:-1] #最後の行のカンマを除去
    json_file.append(']')
    if num_paper == len(papers)-1:
        json_file.append('}')
    else:
        json_file.append('},')

json_file.extend(LINES_END)
path_new = os.path.join(current_dir, OUTPUT_FILE)
with open(path_new, 'w') as f:
    f.write("\n".join(json_file))
print("exporting the json file to : {}".format(path_new))
print("done")

exporting the json file to : YOUR/Current/directory/bibtex_out.json
done
